In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy

# Import Machine Learning Dependencies
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
import io
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
from io import StringIO

In [3]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [4]:
engine = create_engine("sqlite:///final_project")

In [5]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [6]:
results = client.get("xwdj-i9he", limit=1160000)

In [7]:
results_df = pd.DataFrame.from_records(results)

In [8]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_lat,sr_location_lat_long,sr_status_date,sr_location_street_name,sr_created_date,sr_location_street_number,sr_location_city,sr_location_y,sr_location_map_tile,sr_location_zip_code
611506,"10601 FM 2222 RD, AUSTIN, TX 78730",10,Phone,2018-03-03T09:43:45.000,3081333.57152984,Closed,Animal Control - Assistance Request,TRAVIS,492G,-97.84407767,...,30.3931017,"{'latitude': '30.3931017', 'needs_recoding': F...",2018-03-03T09:43:45.000,2222,2018-03-01T14:37:25.000,10601,AUSTIN,10115486.3591601,MD33,78730
54613,"5465 HITCHER BND, AUSTIN, TX 78749",8,Phone,2021-06-16T07:16:44.000,3075921.85324074,Closed,ARR Missed Yard Trimmings/Compost,TRAVIS,642N,-97.86596558,...,30.20869808,"{'latitude': '30.20869808', 'human_address': '...",2021-06-16T07:16:44.000,HITCHER,2021-06-14T18:33:33.000,5465,AUSTIN,10048283.38393480,MC17,78749
704373,"6208 LOST HORIZON DR, AUSTIN, TX 78759",10,Phone,2017-06-12T05:55:23.000,3105005.24766601,Closed,ARR Missed Yard Trimmings /Organics,TRAVIS,464Y,-97.76865839,...,30.4050864,"{'latitude': '30.4050864', 'needs_recoding': F...",2017-06-12T05:55:23.000,LOST HORIZON,2017-06-09T09:45:37.000,6208,AUSTIN,10120383.9981604,MH34,78759
1017400,"CENTRAL COMMERCE DR & GRAND AVENUE PKWY, AUSTI...",NaN,Phone,2015-03-12T10:24:39.000,3138825.88,Closed,Traffic Signal - Maintenance,TRAVIS,437J,-97.65993345,...,30.45595334,"{'latitude': '30.45595334', 'needs_recoding': ...",2015-03-12T10:24:39.000,CENTRAL COMMERCE DR & GRAND AVENUE PKWY,2015-02-10T15:24:20.000,NaN,AUSTIN,10139685.23,MN38,78660
98453,"2107 CAMPFIELD PKWY, AUSTIN, TX 78745",5,Phone,2021-04-01T08:11:37.000,3091479.98027684,Closed,ARR Missed Recycling,TRAVIS,673B,-97.81717968,...,30.19030474,"{'latitude': '30.19030474', 'human_address': '...",2021-04-01T08:11:37.000,CAMPFIELD,2021-03-30T11:52:54.000,2107,AUSTIN,10041940.74074760,ME15,78745
860786,"9010 RESEARCH BLVD SB, AUSTIN, TX 78757",7,Spot311 Interface,2016-04-11T08:56:29.000,3119236.99964783,Closed,Water Waste Report,TRAVIS,495Y,-97.72440552,...,30.37165196,"{'latitude': '30.37165196', 'needs_recoding': ...",2016-04-11T08:56:29.000,RESEARCH,2016-04-08T20:10:42.000,9010,AUSTIN,10108551.9969993,MK31,78757
1138474,"13201 FIELDGATE DR, AUSTIN, TX",NaN,Phone,2014-03-11T18:56:20.000,3143046.16387677,Closed,ARR Missed Recycling,TRAVIS,467X,-97.6479698,...,30.40447353,"{'latitude': '30.40447353', 'needs_recoding': ...",2014-03-11T18:56:20.000,FIELDGATE,2014-03-07T12:55:26.000,13201,AUSTIN,10121058.8801687,MN34,78753
975935,"615 W CESAR CHAVEZ ST, AUSTIN, TX",9,Spot311 Interface,2015-06-03T10:45:29.000,3111908.98384355,Closed,Park Maintenance - Grounds,TRAVIS,584Z,-97.75050538,...,30.2650166,"{'latitude': '30.2650166', 'needs_recoding': F...",2015-06-03T10:45:29.000,CESAR CHAVEZ,2015-05-31T20:03:24.000,615,AUSTIN,10069588.2428107,MH22,78701
581308,"6211 WALKER LN, AUSTIN, TX 78741",3,Phone,2018-11-13T01:02:03.000,3129570.4369983,Closed,Pavement Failure,TRAVIS,616P,-97.69526507,...,30.23870858,"{'latitude': '30.23870858', 'needs_recoding': ...",2018-11-13T01:02:03.000,WALKER,2018-05-15T14:08:18.000,6211,AUSTIN,10060432.150099,ML20,78741
214852,"12506 ORO VALLEY TRL, AUSTIN, TX 78729",6,Phone,2021-02-01T00:03:13.000,3105826.22719887,Closed,Tree Issue ROW,WILLIAMSON,434Y,-97.76506342,...,30.4422479,"{'latitude': '30.4422479', 'human_address': '{...",2021-02-01T00:03:13.000,ORO VALLEY,2020-07-20T19:10:56.000,12506,AUSTIN,10133921.73237850,MH37,78729


In [9]:
len(results_df)

1157049

In [10]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [11]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [12]:
results_df['Request Month'] = pd.DatetimeIndex(results_df['Created Date']).month

In [13]:
results_df['Time of Day'] = pd.DatetimeIndex(results_df['Created Date']).hour

In [14]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [15]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [16]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [17]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [18]:
median = results_df['Total Seconds'].median(axis=0)

In [19]:
conditions = [
    (results_df['Total Seconds'] <= median),
    (results_df['Total Seconds'] > median)
    ]
values = ['Quick Response', 'Slow Response']
results_df['Response Time'] = np.select(conditions, values)

In [20]:
conditions = [
    (results_df['Request Month'] == 1),
    (results_df['Request Month'] == 2),
    (results_df['Request Month'] == 3),
    (results_df['Request Month'] == 4),
    (results_df['Request Month'] == 5),
    (results_df['Request Month'] == 6),
    (results_df['Request Month'] == 7),
    (results_df['Request Month'] == 8),
    (results_df['Request Month'] == 9),
    (results_df['Request Month'] == 10),
    (results_df['Request Month'] == 11),
    (results_df['Request Month'] == 12)
]
values = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
results_df['Request Month'] = np.select(conditions, values)

In [21]:
#categorize the Hour variable
conditions = [
    (results_df['Time of Day'] >= 0) & (results_df['Time of Day'] < 4),
    (results_df['Time of Day'] >= 4) & (results_df['Time of Day'] < 8),
    (results_df['Time of Day'] >= 8) & (results_df['Time of Day'] < 12),
    (results_df['Time of Day'] >= 12) & (results_df['Time of Day'] < 16),
    (results_df['Time of Day'] >= 16) & (results_df['Time of Day'] < 20),
    (results_df['Time of Day'] >= 20) & (results_df['Time of Day'] <= 23)
    ]
values = ['Early Morning', 'Mid Morning', 'Late Morning', 'Afternoon', 'Evening', 'Night']
results_df['Time of Day'] = np.select(conditions, values)

In [22]:
timecheck = results_df['Time of Day'].value_counts()
timecheck

Late Morning     107091
Afternoon         96947
Evening           69938
Night             25225
Mid Morning       21088
Early Morning      6026
Name: Time of Day, dtype: int64

In [23]:
results_df.head()

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_zip_code,Created Date,Closed Date,Request Year,Request Month,Time of Day,Open Time,Open Time delta,Total Seconds,Response Time
300904,"6510 BERKMAN DR, AUSTIN, TX 78723",1,Phone,2020-01-02T18:13:51.000,3130569.45570825,Closed,Loud Commercial Music,TRAVIS,556K,-97.68991552,...,78723,2019-12-31 23:57:21,2020-01-02 18:13:51,2019,December,Night,1 days 18:16:30,1 days 18:16:30,152190.0,Slow Response
300905,"1800 E STASSNEY LN, AUSTIN, TX 78744",2,Spot311 Interface,2020-01-02T18:13:45.000,3109221.74865566,Closed,Loud Commercial Music,TRAVIS,644U,-97.76081753,...,78744,2019-12-31 23:44:51,2020-01-02 18:13:45,2019,December,Night,1 days 18:28:54,1 days 18:28:54,152934.0,Slow Response
300906,"400 E 7TH ST, AUSTIN, TX 78701",9,Web,2020-01-02T07:03:59.000,3115538.40203928,Closed,Street Light Issue- Address,TRAVIS,585T,-97.7389242,...,78701,2019-12-31 23:35:12,2020-01-02 07:03:59,2019,December,Night,1 days 07:28:47,1 days 07:28:47,113327.0,Slow Response
300907,"200 E 15TH ST, AUSTIN, TX 78701",1,Spot311 Interface,2020-01-02T09:06:02.000,3115838.49761388,Duplicate (closed),ARR Dead Animal Collection,TRAVIS,585P,-97.737741,...,78701,2019-12-31 23:17:46,2020-01-02 09:06:02,2019,December,Night,1 days 09:48:16,1 days 09:48:16,121696.0,Slow Response
300908,"200 SAN JACINTO BLVD, AUSTIN, TX 78701",9,Phone,2020-01-02T18:13:44.000,3114645.13713938,Closed,Loud Commercial Music,TRAVIS,585W,-97.74186188,...,78701,2019-12-31 23:03:52,2020-01-02 18:13:44,2019,December,Night,1 days 19:09:52,1 days 19:09:52,155392.0,Slow Response


In [24]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [25]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [26]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [27]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [28]:
training_df = results_df.drop(columns=['sr_location', 'Open Time', 'sr_location_lat_long', 'sr_location_council_district', 'sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_long', 'sr_updated_date', 'sr_location_lat', 'sr_status_date', 'sr_location_street_name', 'sr_created_date', 'sr_location_street_number', 'sr_location_city', 'sr_location_y', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Open Time delta'])

In [29]:
training_df = training_df.drop(columns=['sr_type_desc','sr_number'])

In [30]:
#training_df = training_df.drop(columns=[])

In [31]:
training_df = training_df.rename(columns={'sr_method_received_desc':'Report Method', "sr_location_zip_code":"Zip Code"})

In [32]:
training_df = training_df.rename(columns={'Zip Code':'ZipCode'})

In [33]:
training_df.head(10)

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Total Seconds,Response Time,Department
300904,Phone,78723,2019,December,Night,152190.0,Slow Response,Austin Code
300905,Spot311 Interface,78744,2019,December,Night,152934.0,Slow Response,Austin Code
300906,Web,78701,2019,December,Night,113327.0,Slow Response,Public Works
300907,Spot311 Interface,78701,2019,December,Night,121696.0,Slow Response,Austin Resource Recovery
300908,Phone,78701,2019,December,Night,155392.0,Slow Response,Austin Code
300909,Phone,78745,2019,December,Night,18479378.0,Slow Response,Animal Services
300910,Spot311 Interface,78745,2019,December,Night,155819.0,Slow Response,Austin Code
300911,Phone,78746,2019,December,Night,1097446.0,Slow Response,Austin Resource Recovery
300912,Spot311 Interface,78745,2019,December,Night,117788.0,Slow Response,Animal Services
300913,Spot311 Interface,78660,2019,December,Night,162582.0,Slow Response,Animal Services


In [34]:
#read in census data from csv exports from data.census.gov 
mfi_data = "circle/MFI by Zipcode.csv"
pop_data = "circle/Population by Zipcode.csv"

mfi_df = pd.read_csv(mfi_data)
pop_df = pd.read_csv(pop_data)

In [35]:
#drop 0 index row
pop_df = pop_df.drop(0)

In [36]:
mfi_df = mfi_df.drop(0)

In [37]:
#regex to match just zip code 
pop_df['NAME'] = pop_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [38]:
mfi_df['NAME'] = mfi_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [39]:
pop_df = pop_df.rename(columns={'NAME':'ZipCode','B01003_001E':'TotalPopulation'})

In [40]:
pop_df = pop_df.drop(columns=['GEO_ID','B01003_001M'])

In [41]:
mfi_df = mfi_df.rename(columns={'NAME':'ZipCode','S1903_C03_015E':'MedianFamilyIncome'})

In [42]:
mfi_df = mfi_df.drop(columns=['GEO_ID'])

In [43]:
joined_df = mfi_df.join(pop_df.set_index('ZipCode'), on='ZipCode')

In [44]:
joined_df.head()

,ZipCode,MedianFamilyIncome,TotalPopulation
1,78610,88053,38525
2,78613,115556,81943
3,78617,63405,28930
4,78641,108589,72078
5,78652,103846,5374


In [45]:
training_df = training_df.join(joined_df.set_index('ZipCode'), on='ZipCode')

In [46]:
training_df = training_df.drop(columns = ["Total Seconds"])

In [47]:
training_df.head()

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Response Time,Department,MedianFamilyIncome,TotalPopulation
300904,Phone,78723,2019,December,Night,Slow Response,Austin Code,73162,35725
300905,Spot311 Interface,78744,2019,December,Night,Slow Response,Austin Code,52730,50123
300906,Web,78701,2019,December,Night,Slow Response,Public Works,175993,10848
300907,Spot311 Interface,78701,2019,December,Night,Slow Response,Austin Resource Recovery,175993,10848
300908,Phone,78701,2019,December,Night,Slow Response,Austin Code,175993,10848


In [48]:
training_df.drop(training_df[training_df['MedianFamilyIncome'] == '-'].index, inplace = True)

In [49]:
training_df['MedianFamilyIncome'] = pd.to_numeric(training_df['MedianFamilyIncome'])

In [50]:
eighty_mfi = 57260
onetwenty_mfi = 85891
conditions = [
    (training_df['MedianFamilyIncome'] < eighty_mfi),
    (training_df['MedianFamilyIncome'] >= eighty_mfi) & (training_df['MedianFamilyIncome'] <= onetwenty_mfi),
    (training_df['MedianFamilyIncome'] > onetwenty_mfi)
    ]
values = ['Below Median', 'Median', 'Above Median']
training_df['MedianFamilyIncome'] = np.select(conditions, values)

In [51]:
training_df.head()

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Response Time,Department,MedianFamilyIncome,TotalPopulation
300904,Phone,78723,2019,December,Night,Slow Response,Austin Code,Median,35725
300905,Spot311 Interface,78744,2019,December,Night,Slow Response,Austin Code,Below Median,50123
300906,Web,78701,2019,December,Night,Slow Response,Public Works,Above Median,10848
300907,Spot311 Interface,78701,2019,December,Night,Slow Response,Austin Resource Recovery,Above Median,10848
300908,Phone,78701,2019,December,Night,Slow Response,Austin Code,Above Median,10848


In [52]:
#training_df.rename(columns={'index':'Index'}, inplace =True)
#command = 'Index serial primary key, Report Method text, Zip Code varchar(5), Request Year int, Total Seconds interval, Response Time text, Department text'


training_df.to_sql('database', con=engine, if_exists='replace')

engine.execute("SELECT * FROM database").fetchall()

[(300904, 'Phone', '78723', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Median', '35725'),
 (300905, 'Spot311 Interface', '78744', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Below Median', '50123'),
 (300906, 'Web', '78701', 2019, 'December', 'Night', 'Slow Response', 'Public Works', 'Above Median', '10848'),
 (300907, 'Spot311 Interface', '78701', 2019, 'December', 'Night', 'Slow Response', 'Austin Resource Recovery', 'Above Median', '10848'),
 (300908, 'Phone', '78701', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Above Median', '10848'),
 (300909, 'Phone', '78745', 2019, 'December', 'Night', 'Slow Response', 'Animal Services', 'Median', '61907'),
 (300910, 'Spot311 Interface', '78745', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Median', '61907'),
 (300911, 'Phone', '78746', 2019, 'December', 'Night', 'Slow Response', 'Austin Resource Recovery', 'Above Median', '28608'),
 (300912, 'Spot311 Interface', '78745', 2019, 'De

Machine Learning Model

In [53]:
# Create our features
X = training_df.drop(columns = ["Response Time"])

# Create dummy variables
X = pd.get_dummies(X)

# Create our target
y = training_df[["Response Time"]]

In [54]:
# Check the balance of our target values
y["Response Time"].value_counts()

Quick Response    162934
Slow Response     162914
Name: Response Time, dtype: int64

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [56]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_for_class = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
rand_for_class.fit(X_train, y_train.values.ravel())

BalancedRandomForestClassifier(random_state=1)

In [57]:
# Calculated the balanced accuracy score
y_prediction = rand_for_class.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

0.7523648622324277

In [58]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[29481, 11160],
       [ 9008, 31813]], dtype=int64)

In [59]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                      pre       rec       spe        f1       geo       iba       sup

Quick Response       0.77      0.73      0.78      0.75      0.75      0.56     40641
 Slow Response       0.74      0.78      0.73      0.76      0.75      0.57     40821

   avg / total       0.75      0.75      0.75      0.75      0.75      0.57     81462



In [60]:
# List the features sorted in descending order by feature importance
features = rand_for_class.feature_importances_
sort_features = np.argsort(features)[::-1]

for i in sort_features:
    print("{}: ({})".format(X.columns.values[i], features[i]))

Department_Austin Code: (0.27534870715069115)
Department_Austin Resource Recovery: (0.0840713545222632)
Request Year: (0.053931206067423285)
Department_Austin Transportation: (0.04481108655463687)
Department_Animal Services: (0.032174761800222766)
Department_Public Works: (0.026454956677858438)
Report Method_Spot311 Interface: (0.019196648613027296)
Report Method_Phone: (0.01914698488465218)
Department_Watershed Check: (0.017656759671185546)
Request Month_June: (0.016906862762209412)
Time of Day_Afternoon: (0.0160360766875701)
Time of Day_Night: (0.014949963384031005)
Time of Day_Late Morning: (0.014916445950956026)
Request Month_July: (0.01451196046590466)
Request Month_May: (0.0145038716325222)
Time of Day_Evening: (0.01359591827036297)
Request Month_October: (0.013308620302538595)
Request Month_April: (0.013209043313574257)
Department_Parks and Recreation: (0.012990172467122555)
Time of Day_Early Morning: (0.012907701918980038)
Request Month_August: (0.012621096421265677)
Request Mo